In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CassandraIntegration") \
    .config("spark.cassandra.connection.host", "localhost") \
    .config("spark.cassandra.connection.port", "9042") \
    .config("spark.cassandra.auth.username", "cassandra") \
    .config("spark.cassandra.auth.password", "cassandra") \
    .getOrCreate()

2024-03-16 21:34:59,277 WARN util.Utils: Your hostname, BDS-2023 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
2024-03-16 21:34:59,288 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-03-16 21:35:19,642 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
flights_df = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="flights", keyspace="your_keyspace") \
    .load()


Py4JJavaError: An error occurred while calling o45.load.
: java.lang.ClassNotFoundException: 
Failed to find data source: org.apache.spark.sql.cassandra. Please find packages at
http://spark.apache.org/third-party-projects.html
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToFindDataSourceError(QueryExecutionErrors.scala:443)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:670)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:720)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:210)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:174)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflecti

In [ ]:
# Remover duplicatas
flights_no_duplicates = flights_df.dropDuplicates()

# Remover linhas com valores nulos nas colunas 'ArrDelay' e 'DepDelay'
flights_clean = flights_no_duplicates.dropna(how="any", subset=["ArrDelay", "DepDelay"])


In [ ]:
from pyspark.sql.functions import col

# Supondo que 'flights_clean' é o DataFrame resultante após a limpeza
correlation = flights_clean.stat.corr("DepDelay", "ArrDelay")
print(f"Correlation between departure delay and arrival delay: {correlation}")


2024-03-16 21:50:25,617 WARN netty.NettyRpcEnv: Ignored message: HeartbeatResponse(false)
